In [3]:
from sportsipy.nhl.teams import Teams
from scipy.stats import pearsonr
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# https://towardsdatascience.com/web-scraping-nba-stats-4b4f8c525994
def getGa(abbr,year):
    # for abbr in abbr_list:
    url = 'https://www.hockey-reference.com/teams/' + abbr + '/' + str(year) + '.html'
    html = urlopen(url)
    soup = BeautifulSoup(html,features="lxml")
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    all_stats = [[td.getText() for td in rows[i].findAll('td')]for i in range(len(rows))]
    all_stats2 = [e for e in all_stats if e]
    all_stats3=[x for x in all_stats2 if len(x)>3]
    team_stats = all_stats3[0]
    
    goals_against = team_stats[8]
    return(goals_against)

corrd = {}
for c,year in enumerate(range(2010, 2021)):
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
        G = team.games_played
        Rwin = round(team.wins/team.games_played,3)
        GF = team.goals_for
        GA = int(getGa(team.abbreviation,year))
        statd[i] = {'Team' : team.name,
                    'W' : team.wins,
                    'L' : team.losses,
                    'OTL' : team.overtime_losses,
                    'G' : G,
                    'Rwin' : round(((.5*team.overtime_losses)+team.wins)/G,3),
                    'GF' : GF,
                    'GA' : GA,
                    'Rfor' : round(GF/(GF+GA),3),
                    'GD' : GF-GA,
                    'GD_per' : (GF-GA)/G} 

    df = pd.DataFrame(statd)
    df = df.transpose()
    
    x = list(df['Rwin'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Iwin'] =  index

    x = list(df['Rfor'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Ifor'] =  index
    
    x = list(df['GD'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Idiff'] =  index
     
    rankWinFor, _ = pearsonr(df['Ifor'].tolist(), df['Iwin'].tolist())
    valuWinFor, _ = pearsonr(df['Rfor'].tolist(), df['Rwin'].tolist())
    rankWinDif, _ = pearsonr(df['Idiff'].tolist(), df['Iwin'].tolist())
    valuWinDif, _ = pearsonr(df['GD'].tolist(), df['Rwin'].tolist())
      
    corrd[c] = {'rankWinFor':rankWinFor,
                'valuWinFor':valuWinFor,
                'rankWinDif':rankWinDif,
                'valuWinDif':valuWinDif}

df_corr = pd.DataFrame(corrd)
df_corr = df_corr.transpose()
print('average of each correlation:')
df_corr.mean(axis='index')

average of each correlation:


rankWinFor    0.927254
valuWinFor    0.951212
rankWinDif    0.929044
valuWinDif    0.952503
dtype: float64

In [1]:
from sportsipy.mlb.teams import Teams

corrd = {}
for c,year in enumerate(range(2010, 2021)):
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
                    G = team.games
                    GF = int(G*team.runs)
                    GA = int(G*team.runs_against)
                    statd[i] = {'Team' : team.name,
                    'W' : team.wins,
                    'L' : team.losses,
                    'G' : G,
                    'Rwin' : round(team.wins/G,3),
                    'GF' : GF,
                    'GA' : GA,
                    'Rfor' : round(GF/(GF+GA),3),
                    'GD' : GF-GA,
                    'GD_per' : round((GF-GA)/G,2)}

    df = pd.DataFrame(statd)
    df = df.transpose()
    
    x = list(df['Rwin'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Iwin'] =  index

    x = list(df['Rfor'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Ifor'] =  index
    
    x = list(df['GD'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Idiff'] =  index
    
    rankWinFor, _ = pearsonr(df['Ifor'].tolist(), df['Iwin'].tolist())
    valuWinFor, _ = pearsonr(df['Rfor'].tolist(), df['Rwin'].tolist())
    rankWinDif, _ = pearsonr(df['Idiff'].tolist(), df['Iwin'].tolist())
    valuWinDif, _ = pearsonr(df['GD'].tolist(), df['Rwin'].tolist())
      
    corrd[c] = {'rankWinFor':rankWinFor,
                'valuWinFor':valuWinFor,
                'rankWinDif':rankWinDif,
                'valuWinDif':valuWinDif}

df_corr = pd.DataFrame(corrd)
df_corr = df_corr.transpose()
print('average of each correlation:')
df_corr.mean(axis='index')

average of each correlation:


rankWinFor    0.937692
valuWinFor    0.936748
rankWinDif    0.934896
valuWinDif    0.935279
dtype: float64

In [2]:
from sportsipy.nfl.teams import Teams

corrd = {}
for c,year in enumerate(range(2010, 2021)):
    teams = list(Teams(year))
    statd = {}
    for i,team in enumerate(teams[0:len(teams)]):
        statd[i] = {#'TEA' : team.abbreviation,
                    'Team' : team.name,
                    #'Rank' : team.rank,
                    'W' : team.wins,
                    'L' : team.losses,
                    'GP' : team.games_played,
                    'Rwin' : round(team.wins/team.games_played,4),
                    'GF' : team.points_for,
                    'GA' : team.points_against,
                    'Rfor' : round(team.points_for/(team.points_for+team.points_against),4),
                    'GD' : team.points_for-team.points_against,
                    'GD_per' : (team.points_for-team.points_against)/team.games_played}

    df = pd.DataFrame(statd)
    df = df.transpose()
    
    x = list(df['Rwin'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Iwin'] =  index

    x = list(df['Rfor'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Ifor'] =  index
    
    x = list(df['GD'])
    seq = sorted(x,reverse=True)
    index = [seq.index(v) for v in x]
    df['Idiff'] =  index
    
    rankWinFor, _ = pearsonr(df['Ifor'].tolist(), df['Iwin'].tolist())
    valuWinFor, _ = pearsonr(df['Rfor'].tolist(), df['Rwin'].tolist())
    rankWinDif, _ = pearsonr(df['Idiff'].tolist(), df['Iwin'].tolist())
    valuWinDif, _ = pearsonr(df['GD'].tolist(), df['Rwin'].tolist())
      
    corrd[c] = {'rankWinFor':rankWinFor,
                'valuWinFor':valuWinFor,
                'rankWinDif':rankWinDif,
                'valuWinDif':valuWinDif}

df_corr = pd.DataFrame(corrd)
df_corr = df_corr.transpose()
print('average of each correlation:')
df_corr.mean(axis='index')

average of each correlation:


rankWinFor    0.893465
valuWinFor    0.910221
rankWinDif    0.897721
valuWinDif    0.915753
dtype: float64